# Professional Services Retreat | GenAI Workshop

# Prompts

Each time we make a request to an LLM we do so via a prompt. Here we will look at the three main prompts we use in the Text2Cypher application.

They include
* the agent initialization prompt
* The Cypher generation prompt
* The Cypher results summarization prompt

## Imports

In [1]:
import sys

sys.path.append("../../")
from src.ps_genai_agents.prompts import create_agent_prompt, create_graphqa_chain_cypher_prompt, create_final_summary_prompt_without_lists

## Agent Initialization Prompt

When constructing an agent, we must pass it a prompt that will define it's behavior. This is the responsibility of the `System Message`. 

In this prompt template we also have attributes for `chat_history`, `input` and `agent_scratchpad`. 

We will use `input` to contain the user question.

`agent_scratchpad` will contain any processing steps performed.

We can optionally use `chat_history` to store the chat history.

In [2]:
agent_prompt = create_agent_prompt()
agent_prompt.pretty_print()

================================ System Message ================================


          You are a human assistant and you can retrieve survey response information from Neo4j.
          Use your tools to answer questions.
          If you do not have a tool to answer the question, say so.
          If an ID is present in the query, use Text2Cypher.
          Do not generate Cypher unless it is via the Text2Cypher tool!
          

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


## Cypher Generation Prompt

Since we are using `LangChain`'s Text2Cypher implementation, we will also use `LangChain`'s [`FewShotPromptTemplate`](https://python.langchain.com/docs/how_to/few_shot_examples/) to structure our Cypher generation prompt. 

This requires
* **prefix**: We place our generation instructions here
* **examples**: a list of examples
* **example template**: how each example should be formatted
* **suffix**: This contains our placeholder for `question`
* placeholders for `question` and `schema`

We can see that we also declare some sections 
* `<instructions>...</instructions>`
* `<schema>...</schema>`
* `<samples>...</samples>`

`ps-genai-agents` contains a function that will piece this together for us. We just need to provide the path to the `queries.yml` file.

The `question` and `schema` placeholders will be filled by the `Langchain` Text2Cypher class `GraphCypherQAChain` when making a request. We will explain this class more in future notebooks.


In [3]:
cypher_prompt = create_graphqa_chain_cypher_prompt("../../data/iqs/queries/queries.yml")
cypher_prompt.pretty_print()

You are an expert Neo4j Cypher translator who understands the question in english and convert to Cypher strictly based on the Neo4j Schema provided and following the instructions below:
    <instructions>
    * Use aliases to refer the node or relationship in the generated Cypher query
    * Generate Cypher query compatible ONLY for Neo4j Version 5
    * Do not use EXISTS in the cypher. Use alias when using the WITH keyword
    * Only use SIZE when checking the size of a list
    * Use only Nodes and relationships mentioned in the schema
    * Always enclose the Cypher output inside 3 backticks (```)
    * Always do a case-insensitive and fuzzy search for any properties related search. Eg: to search for a Company name use `toLower(c.name) contains 'neo4j'`
    * Cypher is NOT SQL. So, do not mix and match the syntaxes
    * Only MATCH on the following properties: ["id", "verbatimText"] unless absolutely necessary.
    * Ensure that null results are filtered out before running aggregati

## Summarization Prompt

Our summarization prompt is responsible for converting the raw Cypher result into something more readable. 

Here we are using the summarization prompt that will attempt to return results while avoiding a list format. 

In [5]:
result = [{"animals": ["dogs", "cats", "alligators"]}]
question = "What are some pet options?"
summarization_prompt = create_final_summary_prompt_without_lists(tool_execution_result=result, question=question)
print(summarization_prompt)

Human:
    Fact: [{'animals': ['dogs', 'cats', 'alligators']}]

    * Summarise the above fact as if you are answering this question "What are some pet options?"
    * When the fact is not empty, assume the question is valid and the answer is true
    * Do not return extra text or apologies
    * Just return summary to the user. DO NOT start with "Here is a summary"
    * Don't report empty String results, but include results that are 0 or 0.0.
    * Only use bulletpoints to improve readability. Do not just list the provided Facts. Do not ever use more than a FEW bulletpoints.
    Assistant:
    
